In [1]:
#!conda create -n ML_gpu tensorflow-gpu pytorch-gpu xarray dask matplotlib nb_conda_kernels jupyterlab cudatoolkit cupy esmtools climpred -y
#!pip install git+https://github.com/jacobtomlinson/cupy-xarray.git

In [2]:
!conda list cupy

# packages in environment at /work/mh0727/m300524/conda-envs/ML_gpu:
#
# Name                    Version                   Build  Channel
cupy                      8.6.0            py39h694feb1_0    conda-forge
cupy-xarray               0.1.0+2.g08c5779          pypi_0    pypi


In [3]:
def nvidia_smi():
    import subprocess
    p = subprocess.check_output('nvidia-smi').strip().decode('utf-8')
    return p

## climpred on CPU vs GPU

In [4]:
import xarray as xr
import numpy as np
from climpred.tutorial import load_dataset
from climpred import PerfectModelEnsemble

### CPU

In [6]:
v = "tos"
ds3d = load_dataset("MPI-PM-DP-3D")[v]
ds3d.lead.attrs["unit"] = "years"
control3d = load_dataset("MPI-control-3D")[v]

pm_cpu = PerfectModelEnsemble(ds3d)
pm_cpu = pm_cpu.add_control(control3d)

In [7]:
type(ds3d.data)

numpy.ndarray

In [16]:
%timeit _ = pm_cpu.mean(['x','y'])

24.3 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%timeit skill_cpu = pm_cpu.verify(metric="pearson_r", comparison="m2m", dim=["init", "member"])[v]

667 ms ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### GPU

In [10]:
import cupy_xarray

In [11]:
v = "tos"
ds3d = load_dataset("MPI-PM-DP-3D")[v].as_cupy()
ds3d.lead.attrs["unit"] = "years"
control3d = load_dataset("MPI-control-3D")[v].as_cupy()

pm_gpu = PerfectModelEnsemble(ds3d)
pm_gpu = pm_gpu.add_control(control3d)

/mnt/lustre01/pf/zmaw/m300524/climpred/climpred/utils.py:140: UserWarning: Assuming annual resolution due to numeric inits. Change init to a datetime if it is another resolution.
  warnings.warn(


In [12]:
type(ds3d.data)

cupy.core.core.ndarray

In [15]:
%timeit _ = pm_gpu.mean(['x','y'])

423 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
%timeit skill_gpu = pm_gpu.verify(metric="pearson_r", comparison="m2m", dim=["init", "member"])[v]

64.1 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


> 🚀 GPU has the potential to increase verify by factor 10! 🚀

### GPU Limitations

with {py:meth}`.PerfectModelEnsemble.bootstrap`

In [20]:
pm_gpu.bootstrap(metric="pearson_r", comparison="m2m", dim=["init", "member"], iterations=10)[v]

TypeError: no implementation found for 'numpy.quantile' on types that implement __array_function__: [<class 'cupy.core.core.ndarray'>, <class 'numpy.ndarray'>]